# Inference Testing

In [22]:
import os

import tensorflow as tf
import cv2
from PIL import Image, ImageDraw, ImageFont
from six import BytesIO
import numpy as np

In [56]:
SAVED_MODEL = '/home/jay/projects/tensorflow/workspace/training_demo/exported-models/resnet50_640/saved_model'
SAMPLE_IMAGE = '/home/jay/projects/tf20_vision_samples/cv_images/16076197128-1-0-c.jpg'

# Load saved model and build the detection function
detect_fn = tf.saved_model.load("./trained_model/saved_model")

# Read image
image = cv2.imread("./data/images/image.jpg")
image_copy = image[np.newaxis, ...]

# Compute detections
output = detect_fn(image_copy)

In [57]:
detection_model = tf.saved_model.load(SAVED_MODEL)

In [58]:
# Read image
image = cv2.imread(SAMPLE_IMAGE)
image_copy = image[np.newaxis, ...]

In [59]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path.

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    #img_data = tf.io.gfile.GFile(path, 'rb').read()
    #image = Image.open(BytesIO(img_data))

    image = cv2.imread(path)
    # print ("load path", path, image.shape, image.dtype)
    (im_width, im_height, im_channels) = image.shape
    return image

In [60]:
test_image_dir = '/home/jay/projects/tf20_vision_samples/cv_images/'

test_image_files = []
for f in os.listdir(test_image_dir):
    if os.path.isfile(os.path.join(test_image_dir, f))  and f.endswith('.jpg'):
        test_image_files.append(f)
print ("test image count:", len(test_image_files))

test_images_np = []



test image count: 13


In [61]:

for image_path in test_image_files:
    test_images_np.append(np.expand_dims(load_image_into_numpy_array(os.path.join(test_image_dir, image_path)), axis=0))
    
print ("data shape:", type(test_images_np), len(test_images_np))

data shape: <class 'list'> 13


## Detect

In [62]:
# Again, uncomment this decorator if you want to run inference eagerly
@tf.function
def detect(input_tensor):
  """Run detection on an input image.

  Args:
    input_tensor: A [1, height, width, 3] Tensor of type tf.float32.
      Note that height and width can be anything since the image will be
      immediately resized according to the needs of the model within this
      function.

  Returns:
    A dict containing 3 Tensors (`detection_boxes`, `detection_classes`,
      and `detection_scores`).
  """
  preprocessed_image, shapes = detection_model.preprocess(input_tensor)
  prediction_dict = detection_model.predict(preprocessed_image, shapes)
  return detection_model.postprocess(prediction_dict, shapes)

In [63]:
# Note that the first frame will trigger tracing of the tf.function, which will
# take some time, after which inference should be fast.

label_id_offset = 1
for i in range(len(test_images_np)):
  input_tensor = tf.convert_to_tensor(test_images_np[i], dtype=tf.float32)
  detections = detect(input_tensor)

  plot_detections(
      test_images_np[i][0],
      detections['detection_boxes'][0].numpy(),
      detections['detection_classes'][0].numpy().astype(np.uint32)
      + label_id_offset,
      detections['detection_scores'][0].numpy(),
      category_index, figsize=(15, 20), image_name="gif_frame_" + ('%02d' % i) + ".jpg")

AttributeError: in user code:

    <ipython-input-62-0c245788601c>:16 detect  *
        preprocessed_image, shapes = detection_model.preprocess(input_tensor)

    AttributeError: '_UserObject' object has no attribute 'preprocess'
